### Tarea 6: Opcional (10% del trabajo)

Entrenar el Spacy con el corpus cess-esp para realizar POS tagging en español y obtener las prestaciones del etiquetador en términos de accuracy.

In [2]:
# Importar las bibliotecas necesarias
import nltk
import json
import spacy
from nltk.corpus import cess_esp
from spacy.tokens import Doc
from spacy.training import Example
!pip install es_core_news_sm

# Descargar el corpus CESS-ESP
nltk.download('cess_esp')


[nltk_data] Downloading package cess_esp to /root/nltk_data...
[nltk_data]   Package cess_esp is already up-to-date!


True

In [10]:
# Obtener las oraciones etiquetadas
sentences = cess_esp.tagged_sents()

# Convertir las oraciones al formato de spaCy y dividir en entrenamiento y prueba
def convert_to_spacy_format(sentences):
    data = []
    for sent in sentences:
        words, tags = zip(*sent)
        data.append((" ".join(words), {"words": words, "tags": tags}))
    return data

all_data = convert_to_spacy_format(sentences)

# Dividir en entrenamiento y prueba
split = int(0.8 * len(all_data))
train_data = all_data[:split]
test_data = all_data[split:]

In [11]:
# Cargar el modelo base en español y preparar los datos
nlp = spacy.blank("es")

# Añadir el etiquetador al pipeline
nlp.add_pipe("tagger")

# Añadir las etiquetas al etiquetador
tagger = nlp.get_pipe("tagger")
for _, annotations in train_data:
    for tag in annotations.get("tags"):
        tagger.add_label(tag)

# Entrenar el modelo
optimizer = nlp.begin_training()
for itn in range(10):
    for text, annotations in train_data:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], drop=0.2, sgd=optimizer)


In [12]:
# Evaluar el modelo en el conjunto de prueba
correct = 0
total = 0
for text, annotations in test_data:
    doc = nlp(text)
    predicted_tags = [token.tag_ for token in doc]
    true_tags = annotations["tags"]
    correct += sum(1 for p, t in zip(predicted_tags, true_tags) if p == t)
    total += len(true_tags)

accuracy = correct / total
print(f"Accuracy: {accuracy:.4f}")


Accuracy: 0.6009
